# Clustering aplicado: recomendando músicas com K-Means

**Projeto:** Recomendador de músicas utilizando Machine Learning

>Este projeto foi feito com base em um curso da Alura, Clustering aplicado: recomendando músicas com K-Means, da Formação Machine Learning.

> https://cursos.alura.com.br/formacao-machine-learning

**Sobre os Dados:** Retirados da API do Spotify (https://developer.spotify.com/) e previamente tratados para utilização no curso pela instrutora (https://github.com/sthemonica). Os datasets utilizados não contêm mais todas as variáveis especificadas na API Web.

# Análise preliminar e limpeza dos datasets

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Dados gerais de músicas
dados = pd.read_csv('https://raw.githubusercontent.com/sthemonica/music-clustering/main/Dados/Dados_totais.csv')
# Dados relacionados à gêneros
dados_generos = pd.read_csv('https://raw.githubusercontent.com/sthemonica/music-clustering/main/Dados/data_by_genres.csv')
# Dados relacionados aos anos
dados_anos = pd.read_csv('https://raw.githubusercontent.com/sthemonica/music-clustering/main/Dados/data_by_year.csv')

In [3]:
dados.head(3)

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,speechiness,tempo,artists_song
0,0.285,2000,0.00239,Coldplay,0.429,266773,0.661,0,3AJwUDP919kvQ9QcozQPxg,0.000121,11,0.2340,-7.227,1,Yellow,84,0.0281,173.372,Coldplay - Yellow
1,0.613,2000,0.14300,OutKast,0.843,270507,0.806,1,0I3q5fE6wg7LIfHGngUTnV,0.000000,4,0.0771,-5.946,0,Ms. Jackson,80,0.2690,94.948,OutKast - Ms. Jackson
2,0.400,2000,0.00958,Linkin Park,0.556,216880,0.864,0,60a0Rd6pjrkxjPbaKzXjfq,0.000000,3,0.2090,-5.870,0,In the End,84,0.0584,105.143,Linkin Park - In the End


In [4]:
dados_generos.head(3)

,mode,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key
0,1,21st century classical,0.979333,0.162883,1.602977e+05,0.071317,0.606834,0.3616,-31.514333,0.040567,75.336500,0.103783,27.833333,6
1,1,432hz,0.494780,0.299333,1.048887e+06,0.450678,0.477762,0.1310,-16.854000,0.076817,120.285667,0.221750,52.500000,5
2,1,8-bit,0.762000,0.712000,1.151770e+05,0.818000,0.876000,0.1260,-9.180000,0.047000,133.444000,0.975000,48.000000,7


In [5]:
dados_anos.head(3)

,mode,year,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key
0,1,1921,0.886896,0.418597,260537.166667,0.231815,0.344878,0.205710,-17.048667,0.073662,101.531493,0.379327,0.653333,2
1,1,1922,0.938592,0.482042,165469.746479,0.237815,0.434195,0.240720,-19.275282,0.116655,100.884521,0.535549,0.140845,10
2,1,1923,0.957247,0.577341,177942.362162,0.262406,0.371733,0.227462,-14.129211,0.093949,114.010730,0.625492,5.389189,0


In [6]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20311 entries, 0 to 20310
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   valence           20311 non-null  float64
 1   year              20311 non-null  int64  
 2   acousticness      20311 non-null  float64
 3   artists           20311 non-null  object 
 4   danceability      20311 non-null  float64
 5   duration_ms       20311 non-null  int64  
 6   energy            20311 non-null  float64
 7   explicit          20311 non-null  int64  
 8   id                20311 non-null  object 
 9   instrumentalness  20311 non-null  float64
 10  key               20311 non-null  int64  
 11  liveness          20311 non-null  float64
 12  loudness          20311 non-null  float64
 13  mode              20311 non-null  int64  
 14  name              20311 non-null  object 
 15  popularity        20311 non-null  int64  
 16  speechiness       20311 non-null  float6

### Anos

In [7]:
dados['year'].unique()

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])

In [8]:
dados_anos['year'].unique()

array([1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931,
       1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939, 1940, 1941, 1942,
       1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953,
       1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964,
       1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975,
       1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986,
       1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997,
       1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008,
       2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
       2020])

* Os dois datasets possuem abrangências diferentes de anos, fazendo com que não haja um cruzamento completo entre os dois.

* Portanto, o dataset dados_anos pode ser reduzido para abranger os anos 2000 a 2020.

## Verificação de dados nulos

### Dados

In [9]:
dados.isnull().sum()

valence             0
year                0
acousticness        0
artists             0
danceability        0
duration_ms         0
energy              0
explicit            0
id                  0
instrumentalness    0
key                 0
liveness            0
loudness            0
mode                0
name                0
popularity          0
speechiness         0
tempo               0
artists_song        0
dtype: int64

In [10]:
dados.isna().sum()

valence             0
year                0
acousticness        0
artists             0
danceability        0
duration_ms         0
energy              0
explicit            0
id                  0
instrumentalness    0
key                 0
liveness            0
loudness            0
mode                0
name                0
popularity          0
speechiness         0
tempo               0
artists_song        0
dtype: int64

### Dados relacionados ao gênero

In [11]:
dados_generos.isnull().sum()

mode                0
genres              0
acousticness        0
danceability        0
duration_ms         0
energy              0
instrumentalness    0
liveness            0
loudness            0
speechiness         0
tempo               0
valence             0
popularity          0
key                 0
dtype: int64

In [12]:
dados_generos.isna().sum()

mode                0
genres              0
acousticness        0
danceability        0
duration_ms         0
energy              0
instrumentalness    0
liveness            0
loudness            0
speechiness         0
tempo               0
valence             0
popularity          0
key                 0
dtype: int64

### Dados relacionados ao ano

In [13]:
dados_anos.isnull().sum()

mode                0
year                0
acousticness        0
danceability        0
duration_ms         0
energy              0
instrumentalness    0
liveness            0
loudness            0
speechiness         0
tempo               0
valence             0
popularity          0
key                 0
dtype: int64

In [14]:
dados_anos.isna().sum()

mode                0
year                0
acousticness        0
danceability        0
duration_ms         0
energy              0
instrumentalness    0
liveness            0
loudness            0
speechiness         0
tempo               0
valence             0
popularity          0
key                 0
dtype: int64

## Remoção de features

* Algumas das features serão removidas por não criarem uma relação relevante entre as músicas em uma recomendação.
  * Serão elas: explicity, key, mode

In [15]:
dados = dados.drop(['explicit', 'key', 'mode'], axis = 1)
dados.head(3)

,valence,year,acousticness,artists,danceability,duration_ms,energy,id,instrumentalness,liveness,loudness,name,popularity,speechiness,tempo,artists_song
0,0.285,2000,0.00239,Coldplay,0.429,266773,0.661,3AJwUDP919kvQ9QcozQPxg,0.000121,0.2340,-7.227,Yellow,84,0.0281,173.372,Coldplay - Yellow
1,0.613,2000,0.14300,OutKast,0.843,270507,0.806,0I3q5fE6wg7LIfHGngUTnV,0.000000,0.0771,-5.946,Ms. Jackson,80,0.2690,94.948,OutKast - Ms. Jackson
2,0.400,2000,0.00958,Linkin Park,0.556,216880,0.864,60a0Rd6pjrkxjPbaKzXjfq,0.000000,0.2090,-5.870,In the End,84,0.0584,105.143,Linkin Park - In the End


In [16]:
dados.shape

(20311, 16)

In [17]:
dados_generos = dados_generos.drop(['key', 'mode'], axis = 1)
dados_generos.head(3)

,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,21st century classical,0.979333,0.162883,1.602977e+05,0.071317,0.606834,0.3616,-31.514333,0.040567,75.336500,0.103783,27.833333
1,432hz,0.494780,0.299333,1.048887e+06,0.450678,0.477762,0.1310,-16.854000,0.076817,120.285667,0.221750,52.500000
2,8-bit,0.762000,0.712000,1.151770e+05,0.818000,0.876000,0.1260,-9.180000,0.047000,133.444000,0.975000,48.000000


In [18]:
dados_anos = dados_anos.drop(['key', 'mode'], axis = 1)
dados_anos.head(3)

,year,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,1921,0.886896,0.418597,260537.166667,0.231815,0.344878,0.205710,-17.048667,0.073662,101.531493,0.379327,0.653333
1,1922,0.938592,0.482042,165469.746479,0.237815,0.434195,0.240720,-19.275282,0.116655,100.884521,0.535549,0.140845
2,1923,0.957247,0.577341,177942.362162,0.262406,0.371733,0.227462,-14.129211,0.093949,114.010730,0.625492,5.389189


## Remoção de entradas

* Não serão necessários todos os anos presentes no dataset **dados_anos**.

In [19]:
filtro = dados_anos['year'] >= 2000
dados_anos = dados_anos[filtro]
dados_anos['year'].unique()

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])

In [20]:
dados_anos.reset_index()

,index,year,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,79,2000,0.289323,0.590918,242724.642638,0.625413,0.101168,0.197686,-8.247766,0.089205,118.999323,0.559475,46.684049
1,80,2001,0.286842,0.583318,240307.796010,0.626986,0.107214,0.187026,-8.305095,0.089182,117.765399,0.541479,48.750125
2,81,2002,0.282624,0.576160,239503.283000,0.641270,0.088048,0.193911,-7.686640,0.084308,119.239738,0.542397,48.655500
3,82,2003,0.256471,0.575763,244670.575230,0.660165,0.083049,0.196976,-7.485545,0.093926,120.914622,0.530504,48.626407
4,83,2004,0.280559,0.567680,237378.708037,0.648868,0.077934,0.202199,-7.601655,0.094239,121.290346,0.524489,49.273143
5,84,2005,0.255764,0.572281,237229.588205,0.653209,0.090194,0.190082,-7.466159,0.093334,121.617967,0.532531,50.953333
6,85,2006,0.279986,0.568230,234042.914359,0.650326,0.077701,0.188289,-7.265501,0.085847,121.798615,0.520028,51.313846
7,86,2007,0.254081,0.563414,241049.962564,0.668305,0.072957,0.196127,-7.044536,0.084347,124.087516,0.516794,51.075897
8,87,2008,0.249192,0.579193,240107.315601,0.671461,0.063662,0.198431,-6.843804,0.077356,123.509934,0.527542,50.630179
9,88,2009,0.261929,0.564190,238140.013265,0.670749,0.075872,0.205252,-7.046015,0.085458,123.463808,0.507170,51.440816


# Análises gráficas

* Plotly
https://plotly.com/python/

In [21]:
import plotly.express as px
import plotly.graph_objects as go

## Variável *loudness*

* **Loudness** é o volume médio da faixa em decibéis (dB). É útil para comparar o volume relativo entre as faixas.

In [22]:
fig = px.line(dados_anos, x = 'year', y = 'loudness', markers = True,
              title = "Variação do loudness conforme os anos")
fig.show()

## Todas as variáveis ao longo dos anos

In [23]:
fig = go.Figure()
fig.update_layout(title = "Características das músicas através dos anos")

fig.add_trace(go.Scatter(x=dados_anos['year'], y=dados_anos['acousticness'], name='Acousticness'))
fig.add_trace(go.Scatter(x=dados_anos['year'], y=dados_anos['valence'], name='Valence'))
fig.add_trace(go.Scatter(x=dados_anos['year'], y=dados_anos['danceability'], name='Danceability'))
fig.add_trace(go.Scatter(x=dados_anos['year'], y=dados_anos['energy'], name='Energy'))
fig.add_trace(go.Scatter(x=dados_anos['year'], y=dados_anos['instrumentalness'], name='Instrumentalness'))
fig.add_trace(go.Scatter(x=dados_anos['year'], y=dados_anos['liveness'], name='Liveness'))
fig.add_trace(go.Scatter(x=dados_anos['year'], y=dados_anos['speechiness'], name='Speechiness'))

fig.show()

# Correlações / Mapa de Calor

In [24]:
fig = px.imshow(dados.corr(), text_auto = True)
fig.show()

# Variáveis numéricas

Citação direta do curso
>Quando estamos lidando com algoritmos de machine learning, alguns algoritmos aprendem melhor quando utilizamos variáveis numéricas dimensionadas em um intervalo padrão. O algoritmo usado no curso, por exemplo, é um dos que utilizam as distâncias para fazer os cálculos.

* **Normalização**

  * Dimensiona os valores de cada feature para um intervalo de 0 a 1.
  
  * $y = \frac{x - min}{max - min}$

  * Quando uma nova entrada é adicionada, ela não estará normalizada.

  * **MinMaxScaler** https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

* **Padronização**

  * Dimensiona os valores das features para que a média seja 0 e o desvio padrão seja 1.

  * $y = \frac{x - média}{Desvio Padrão}$

  * **StandardScaler** https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

  * Assume que os dados têm distribuição gaussiana, em forma de sino, com média e desvio padrão bem comportados.

* No geral, quando os dados têm distribuição Normal, utiliza-se a Padronização.

# Clusterização por gênero

* Agrupa os dados conforme a semelhança entre eles.

* **Algoritmo K-Means**
  * Organiza os dados em k clusters. 
  
(mais explicações sobre o K-Means abaixo)

## Variáveis não numéricas

In [25]:
dados_generos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2973 entries, 0 to 2972
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   genres            2973 non-null   object 
 1   acousticness      2973 non-null   float64
 2   danceability      2973 non-null   float64
 3   duration_ms       2973 non-null   float64
 4   energy            2973 non-null   float64
 5   instrumentalness  2973 non-null   float64
 6   liveness          2973 non-null   float64
 7   loudness          2973 non-null   float64
 8   speechiness       2973 non-null   float64
 9   tempo             2973 non-null   float64
 10  valence           2973 non-null   float64
 11  popularity        2973 non-null   float64
dtypes: float64(11), object(1)
memory usage: 278.8+ KB


In [26]:
dados_generos.head()

,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,21st century classical,0.979333,0.162883,1.602977e+05,0.071317,0.606834,0.361600,-31.514333,0.040567,75.336500,0.103783,27.833333
1,432hz,0.494780,0.299333,1.048887e+06,0.450678,0.477762,0.131000,-16.854000,0.076817,120.285667,0.221750,52.500000
2,8-bit,0.762000,0.712000,1.151770e+05,0.818000,0.876000,0.126000,-9.180000,0.047000,133.444000,0.975000,48.000000
3,[],0.651417,0.529093,2.328809e+05,0.419146,0.205309,0.218696,-12.288965,0.107872,112.857352,0.513604,20.859882
4,a cappella,0.676557,0.538961,1.906285e+05,0.316434,0.003003,0.172254,-12.479387,0.082851,112.110362,0.448249,45.820071


In [27]:
#dados_generos['genres'].value_counts().sum()
len(dados_generos['genres'].unique())

2973

* Cada entrada tem o seu próprio gênero, ou seja, todos são valores únicos.

* Com todos os valores sendo únicos, é possível que a característica não seja útil para o modelo.

* Quando há subcategorias em formato não numérico, que o modelo não conseguiria usas nos cálculos, pode-se transformar esta feature em um **dummie**, que cria uma coluna para cada subcategoria com valores indicando o pertencimento ou não de cada entrada à mesma.
  * Neste caso, seriam criadas novas 2973 colunas, o que não seria interessante. Os valores desta coluna serão guardados para possível uso posterior.

In [28]:
#coluna_generos = dados_generos['genres']
dados_generos_numericos = dados_generos.drop('genres', axis = 1)
dados_generos_numericos.head(3)

,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,0.979333,0.162883,1.602977e+05,0.071317,0.606834,0.3616,-31.514333,0.040567,75.336500,0.103783,27.833333
1,0.494780,0.299333,1.048887e+06,0.450678,0.477762,0.1310,-16.854000,0.076817,120.285667,0.221750,52.500000
2,0.762000,0.712000,1.151770e+05,0.818000,0.876000,0.1260,-9.180000,0.047000,133.444000,0.975000,48.000000


## Pipeline

* O método Pipeline do **sklearn.pipeline** aplica uma sequência de transformações até o resultado final.

* Será definido um passo-a-passo com vários módulos do próprio sklearn para que o processo seja automatizado.

https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

In [29]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

* Há features com margens de valores muito diferentes umas das outras, portanto o primeiro passo será normalizar os dados utilizando o **StandarScaler**.

* O segundo passo é a redução de dimensionalidade com um método de decomposição, utilizando o **PCA**.
  * O PCA (análise de componentes principais) executa uma série de análises matemáticas para diminuir a quantidade de colunas do dataset até uma quantidade desejada. Diminuir muito esta quantidade pode prejudicar o quanto o modelo será capaz de explicar os dados.

In [30]:
SEED = 27
np.random.seed(27)

pca_pipeline = Pipeline([ ('scaler', StandardScaler()), ('PCA', PCA(n_components = 2, random_state = SEED)) ])
genre_embedding_pca = pca_pipeline.fit_transform(dados_generos_numericos)

projection = pd.DataFrame(columns = ['x', 'y'], data = genre_embedding_pca)
projection

,x,y
0,5.910268,-0.011146
1,2.787093,4.498483
2,-0.757538,-1.258495
3,1.020521,-0.931690
4,0.753911,-0.999861
...,...,...
2968,-0.475136,-0.017126
2969,-3.026756,-1.304983
2970,-0.832511,-1.089726
2971,1.774790,-3.695233


* Esta tabela informa as "posições" onde cada entrada se encontra depois da transformação para duas dimensões. Um espaço de 11 dimensões foi projetado para um de 2 dimensões para facilitar a visualização.

* Estes pontos serão agrupados na técnica de clusterização e cada categoria poderá ser visualizada em um gráfico de duas dimensões.

## K-Means

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

* Organiza os dados em k clusters. 

* São definidos k centróides (aleatórios), os dados são adicionados a um gráfico, então os centróides vão sendo ajustados para que os clusters descrevam os dados da melhor forma, e distanciando-os o máximo possível.

* A semelhança entre pontos é calculada através da distância euclidiana. Quanto menor a distância, maior a semelhança.

* Os centróides são os pontos dos quais cada cluster é construído em volta.

* Demonstração de como o K-Means funciona https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/

* Não consegue resolver problemas **escalas diferentes**, por isso é necessária a normalização (StandardScaler), e não consegue agrupar bem dados com **alta dimensionalidade**, por isso é feita a redução de dimensionalidade (PCA).
  * Quando há muitas features, podem haver muitos pares de pontos com distâncias muito semelhantes, atrapalhando um agrupamento significativo.

In [31]:
from sklearn.cluster import KMeans

In [36]:
# verbose -> gera explicações sobre o funcionamento do modelo

kmeans_pca = KMeans(n_clusters = 5, verbose = False, random_state = SEED)
kmeans_pca.fit(projection)

# adicionando os clusters calculados na tabela de gêneros
dados_generos['cluster_pca'] = kmeans_pca.predict(projection)

# adicionando gêneros e clusters na tabela de projeção
projection['cluster_pca'] = kmeans_pca.predict(projection)
projection['genres'] = dados_generos['genres']

projection

,x,y,cluster_pca,genres
0,5.910268,-0.011146,3,21st century classical
1,2.787093,4.498483,3,432hz
2,-0.757538,-1.258495,2,8-bit
3,1.020521,-0.931690,1,[]
4,0.753911,-0.999861,1,a cappella
...,...,...,...,...
2968,-0.475136,-0.017126,0,zolo
2969,-3.026756,-1.304983,2,zouglou
2970,-0.832511,-1.089726,2,zouk
2971,1.774790,-3.695233,1,zurich indie


In [37]:
fig = px.scatter(
    projection,
    x = 'x', y = 'y',
    color = 'cluster_pca',
    hover_data = ['x', 'y', 'genres']
)
fig.show()

## Avalição

* Uma das formas de avaliar se uma clusterização foi bem feita é ver o quanto, em porcentagem, as variáveis estão sendo explicadas dentro dos clusters, utilizando o *explanied_variance_ratio* do PCA.

In [40]:
pca_pipeline[1].explained_variance_ratio_

array([0.34986105, 0.14284565])

* Neste caso, a variável x explica 34.98% dos dados e a variável y 14.28%, somando-se 49.27% (porcentagens aproximadas).

* A porcentagem próxima a 50% foi considerada ravoável por enquanto (para das segmento ao curso).

* *explained_variance_* define quantas variáveis do dataset pré-projeção estão sendo explicadas por cada variável pós projeção. 

In [42]:
pca_pipeline[1].explained_variance_

array([3.84976644, 1.57183087])

* No total, estão sendo explicadas aproximadamente 5.42 features, o original continha 11.